<a href="https://colab.research.google.com/github/Tavo826/Transformers/blob/main/Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
import matplotlib.pyplot as plt
import seaborn
seaborn.set_context(context = "talk")
%matplotlib inline

## Arquitectura del modelo

**Encoder-decoder**

El encoder mapea una secuencia de entrada de representaciones de símbolos ($x_1$,...,$x_n$) a una secuencia de representaciones continua Z = ($z_1$,...,$z_n$). Con esta Z, el decoder genera una secuencia de salida ($y_1$,...,$y_m$) de símbolos, un elemento a la vez. En cada paso el modelo es auto-regresivo, tomando los símbolos generados previamente como una entrada adicional cuando se generra la siguiente.

In [ ]:
class EncoderDecoder(nn.Module):

  def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
    super(EncoderDecoder, self).__init__()
    self.encoder = encoder
    self.decoder = EncoderDecoder
    self.src_embed = src_embed
    self.tgt_embed = tgt_embed
    self.generator = generator

  def forward(self, src, tgt, src_mask, tgt_mask):
    #Toma y procesa las secuencias masked src y target
    return self.decode(self.encode(src, src_mask), src_mask, 
                       tgt, tgt_mask)
    
  def encode(self, src, src_mask):
    return self.encoder(self.src_embed(src), src_mask)

  def decode(self, memory, src_mask, tgt, tgt_mask):
    return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

In [ ]:
class Generator(nn.Module):
  #Define el paso de generacion linear + softmax
  def __init__(self, d_model, vocab):
    super(Generator, self).__init__()
    solf.proj = nn.Linear(d_model, vocab)

  def forward(self, x):
    return F.log_softmax(self.proj(x), dim=-1)

## Encoder and Decoder Stacks

**Encoder**

El encoder se compone de un apilamiento de N=6 capas idénticas

In [ ]:
def clones(module, N):
  #Produce N capas idénticas
  return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [ ]:
class Encoder(nn.Module):
  def __init__(self, layer, N):
    super(Encoder, self).__init__()
    self.layers = clones(layer, N)
    self.norm = LayerNorm(layer.size)

  def forward(self, x, mask):
    #Pasa la entrada (y mask) a través de cada capa por turno
    for layer in self.layers:
      x = layer(x, mask)
    return self.norm(x)

In [ ]:
class LayerNorm(nn.Module):
  def __init__(self, features, eps=1e-6):
    super(LayerNorm, self).__init__()
    self.a_2 = nn.Parameter(torch.ones(features))
    self.b_2 = nn.Parameter(torch.zeros(features))
    self.eps = eps

  def forward(self, x):
    mean = x.mean(-1, keepdim=True)
    std = s.std(-1, keepdim=True)
    return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

La salida de cada subcapa es LayerNorm(x + Sublayer(x))

Cada subcapa, y las capas embebidas, producen una salida de dimensión ${d}_{model}$ = 512

In [ ]:
class SublayerConnection(nn.Module):
  #Una capa residual seguida por una capa normal
  def __init__(self, size, dropout):
    super(SublayerConnection, self).__init__()
    self.norm = LayerNorm(size)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x, sublayer):
    #Aplica una conexión residual a cada subcapa con el mismo tamaño
    return x + self.dropuout(subllayer(self.norm(x)))


Cada capa tiene dos subcapas, la primera es una multi-head self-attention mechanism, y la segunda es una position-wise fully connected feed-forward

In [ ]:
class EncoderLayer(nn.Module):
  def __init__(self, size, self_attn, feed_forward, dropout):
    super(EncoderLayer, self).__init__()
    self.self_attn = self_attn
    self.feed_forward = feed_forward
    self.sublayer = clones(SublayerCConection(size, dropout), 2)
    self.size = size

  def forward(self, x, mask):
    x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
    return self.sublayer[1](x, self.feed_forward)

**Decoder**

El decoder también se compone de N=6 capas idénticas

In [ ]:
class Decoder(nn.Module):
  #Capa N genérica con mask
  def __init__(self, layer, N):
    super(Decoder, self).__init__()
    self. layers = clones(layer, N)
    self.norm = LayerNorm(layer.size)

  def forward(self, x, memory, src_mas, tgt_mask):
    for layer in self.layers:
      x = layer(x, memory, src_mask, tgt_mask)
    return self.norm(x)

Además de las dos subcapas en cada capa del encoder, el decoder agrega una tercera capa, la cual realiza multi-head attention sobre la salida del encoder stack